In [1]:
projecttitle = 'LanguageMVPA'
import sys
import os
if sys.platform == 'darwin':
    sys.path.append(os.path.join("/Users", "njchiang", "GitHub", "task-fmri-utils"))
    sys.path.append(os.path.join("/Users", "njchiang", "GitHub", "task-fmri-utils", "code-init"))
    sys.path.append(os.path.join("/Volumes", "JEFF", "UCLA", projecttitle, "code"))
else:
    sys.path.append(os.path.join("D:\\", "GitHub", "task-fmri-utils"))
    sys.path.append(os.path.join("D:\\", "fmri", projecttitle, "code"))

In [2]:
from project_code import projectutils as pu
from fmri_core import projectanalysis as pa
import logging

In [3]:
paths, sublist, contrasts = pu.initpaths()
con = None
debug = False
write = False
roi = 'grayMatter'
dstype = 'raw'
r = 4  # searchlight radius
sg_params = [49, 2]

Initializing...
I/O error:


In [7]:
# need a trial_type attribute
sublist={'LMVPA001': ["Run1"]}
print "Mask: " + str(roi)
print "Full Model: " + str(con)
print "Searchlight Radius: " + str(r)
print "Write results: " + str(write)
logging.basicConfig(level=logging.DEBUG)

Mask: grayMatter
Full Model: None
Searchlight Radius: 4
Write results: False


In [ ]:
mc_params = pu.loadmotionparams(paths, sublist)


In [ ]:
for s in sublist.keys():
    if dstype == "LSS" or dstype == "LSA":
        rds = pu.preprocess_betas(paths, sub, btype="LSS", c="trial_type", roi=roi, z=True)
    else:
        ds, events = pu.preprocess_data(paths, sublist, s, 
                                        filter_params=sg_params, roi=roi, z=True)
        design_kwargs = {'add_regs': mc_params[s], 'hrf_model': 'canonical'}
        evds = pa.beta_extract(ds, events, design_kwargs=design_kwargs)
        rds = pu.replacetargets(evds, contrasts, thisContrast)
        rds = fds[fds.targets != '0']
    writeopts = {'outdir': os.path.join('multivariate', 'searchlight'), 
                 'sub': s, 'roi': roi,  'con': con}
    sorted_rds = pu.sortds(rds, c='trial_type')
    pa.searchlight(paths, ds, clf=None, cv=None, writeopts=writeopts)

In [ ]:
for s in sublist.keys():
    ds, events = pu.preprocess_data(paths, sublist, s, 
                                    filter_params=sg_params, roi=roi, z=True)
    design_kwargs = {'add_regs': None, 'hrf_model': 'canonical', 'drift_model': 'blank'}
    
    rds, des = pu.preprocess_encoding(ds=ds, events=events, c=con, 
                                      mp=mc_params[s], design_kwargs=design_kwargs)
    des.show()
    writeopts = {'outdir': os.path.join('multivariate', 'encoding'), 
                 'sub': s, 'roi': roi, 'con': con}
    bsargs = {'part_attr': 'chunks', 'mode': 'test', 'single_alpha': True, 'normalpha': False,
              'nboots': 50, 'corrmin': .2, 'singcutoff': 1e-10, 'joined': None, 'plot': False, 'use_corr': True}
    pa.encoding(paths, ds=rds, des=des, c=con, 
                chunklen, nchunks, alphas=None, 
                writeopts=None, bsargs=None)